In [ ]:
import matplotlib.colors as mcolors
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from interpreto.attributions.methods import OcclusionExplainer
from interpreto.commons.granularity import GranularityLevel
from interpreto.visualizations.attributions.classification_highlight import MultiClassAttributionVisualization

/home/fanny.jourdan/dev/interpreto/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
tokenizer_name = [
    "/data/models/notams/distilbert_tokenizer_no-preproc",
    "/data/models/notams/distilbert_tokenizer_preproc",
]
model_name = [
    "/data/models/notams/distillbert_base_trained-6ep_01-2025",
    "/data/models/notams/distillbert_base_trained-8ep_no-preproc_base-token_03-2025",
    "/data/models/notams/distillbert_base_trained-8ep_preproc_base-token_03-2025",
    "/data/models/notams/distillbert_base_trained-8ep_preproc_token-trained_03-2025",
]

model = AutoModelForSequenceClassification.from_pretrained(model_name[0], num_labels=13)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name[0])

explainer = OcclusionExplainer(model=model, batch_size=4, tokenizer=tokenizer, granularity_level=GranularityLevel.WORD)

In [ ]:
attribution_outputs = explainer.explain(model_inputs=["I love this movie", "This movie is terrible"], targets=[1, 0])

ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as pad_token (tokenizer.pad_token = tokenizer.eos_token e.g.) or add a new pad token via tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [ ]:
viz = MultiClassAttributionVisualization(
    attribution_output_list=attribution_outputs,
    class_colors=[mcolors.to_rgb("green"), mcolors.to_rgb("blue")],
    class_names=["class1", "class2"],
    css=".common-word-style {font-size: 1.5em}",
)
viz.display()
